# Script Micom run2
Version GitHub 01

**Running Micom**

Micom v0.35.0  
qiime2-amplicon-2024.2  

########################################################  
By Torben Kuehnast, torben.kuehnast@gmail.com, 2024


In [ ]:
import os
import pandas as pd
import glob
import contextlib
import io
import pickle


from bs4 import BeautifulSoup
from fnmatch import fnmatch
from datetime import datetime

from micom import show_versions
from micom.workflows import build
from micom.workflows import tradeoff
from micom.workflows import grow
from micom.stats import compare_groups
from micom.measures import production_rates
from micom.viz import plot_tradeoff



In [ ]:
#############################################################################################
# Insert files
#############################################################################################

# ---> INSERT project name, a unique name which will be prefixed to all your output files
project = 'project_name'


# ---> INSERT current running version, also prefixed to all your output files
version_nr = 'version'

pro_ver = project+"_"+version_nr



# ---> INSERT the basic directory
# Herein, data can be provided and accessed
# Also, a new folder will be created based on your project name and version, where all output files will be stored (working directory)

basic_dir = '/home/basic_directory'

working_dir = os.path.join(basic_dir, pro_ver)

if not os.path.exists(working_dir):
    os.makedirs(working_dir)
    print(f"Folder {working_dir} was created.")
else:
    print(f"Folder {working_dir} already exists.")
print("Output folder:", working_dir)




# ---> INSERT TAS/taxonomy and abundance from all 8 samples

# Important information:

# The columns of this CSV file need to contain the following headers:
# kingdom;phylum;class;order;family;genus;species;sample_id;abundance
# All other column will not be taken into account

filepath_crop_fmtas = '/home/project_final_micom_tas.csv'
data = pd.read_csv(filepath_crop_fmtas, sep=",")




# ----> INSERT 2 conditions/day 1 vs day 13 in cachexia

# Important information:

# The columns need to be as follows:
# "sample;condition"
# For each sample, you provide the condition's name, like
# TK_137;day01_healthy
# TK_033;day13_sick
# Add all samples you have with the respective conditions
# It will be used to group your samples according to the conditions, to later on look at them differentially

filepath_condition = '/home/condition_file.csv'
data_condition = pd.read_csv(filepath_condition, sep=";")


In [ ]:
# ----> INSERT CPU power used. 1 sample = 1 CPU
cpus_used = int(39)



# ----> INSERT Model catalog. Like McMurGut 1.1

# Important information:
# Make sure the metabolic IDs are the same in the 1) model catalog and 2) the diet file!
# McMurGut is based on gapseq, which is based on the ModelSEED nomenclature, like metabolite-id = "cpd00001" for water.

test_db = '/home/MCMG754_genus.qza'



# ---> INSERT the medium you are using
# Herein, the medium was selfmade, based on the ssniff-diet and completed with Micoms complete_db_medium() function
# based on McMurGut 1.1

test_medium = '/home/ssniff_MCMG754_v04_diet.qza'




# ---> INSERT the full list of all metabolites in THEORY available to gapseq models
# This is needed to transform cpd00001 into more meaningful metabolic IDs (--> H2O) at the end of the script
# and to provide additional information in the middle of the script. Example:
# "id;name"
# "cpd00001;H2O_MNXM2"
# "cpd00002;ATP_MNXM3"
# "cpd00003;NAD_MNXM8"
# It has to contain the "id" and "name" header.

filepath_allmetab = '/home/all_gapseq_metabolites_756_compgr.csv'
allmetab = pd.read_csv(filepath_allmetab, sep=";")


# ---> INSERT cutoff value of abundance for taxa which will no longer taken into simulations (very low abundant ones)
cutoff_value = float(0.0001)

#Minimal coefficient,  See Micom publication for details.
mincoef_value = float(1e-3)

#############################################################################################
#############################################################################################
#############################################################################################
model_folder = str('models_'+pro_ver)
print("model_folder will be: ", model_folder)

print("---------------------------------------------------------")
print("Project name and version:", pro_ver)
print("---------------------------------------------------------")
os.chdir(working_dir)
import os
print("Working directory:", os.getcwd())
print("---------------------------------------------------------")
print("filepath_crop_fmtas:", filepath_crop_fmtas)
print("---------------------------------------------------------")
print("data in filepath_crop_fmtas:", data)
print("---------------------------------------------------------")
print("filepath_condition:", filepath_condition)
print("---------------------------------------------------------")
print('data_condition in filepath_condition:', data_condition)
print("---------------------------------------------------------")
print("CPUS: ", cpus_used)
print("---------------------------------------------------------")
print("Model catalog/test_db", test_db)
print("---------------------------------------------------------")
print("medium filepath:", test_medium)
print("---------------------------------------------------------")
medium = load_qiime_medium(test_medium)
print("medium: ", medium)
print("---------------------------------------------------------")
print("filepath_allmetab, filepath to list of gapseq metabolites:", filepath_allmetab)
print("---------------------------------------------------------")
print("gapseq metabolites from filepath_allmetab: ", allmetab)
print("---------------------------------------------------------")
print("cutoff_value: ", cutoff_value)
print("---------------------------------------------------------")
print("mincoef_value: ", mincoef_value)
print("---------------------------------------------------------")


In [ ]:
from micom import show_versions
import contextlib
import io

def capture_output(function):
    # Create a StringIO object to capture the output
    output = io.StringIO()
    # Use contextlib to redirect stdout to the StringIO object
    with contextlib.redirect_stdout(output):
        function()
    # Get the captured output
    return output.getvalue()

# Capture the output of show_versions
output_text = capture_output(show_versions)
print(output_text)


# Write the output to a text file
with open(f"{pro_ver}_versions.txt", 'w') as f:
    f.write(output_text)

print(f"{pro_ver}_versions.txt saved.")




In [ ]:
# Make sure you can see all the columns separated nicely. If not, Jupyter may not be able to distinguish them properly!
data

In [ ]:
data_condition

In [ ]:
cpus_used

In [ ]:
allmetab

In [ ]:
cutoff_value

In [ ]:
mincoef_value

In [ ]:
medium

In [ ]:
# Collapsing abundances of equal taxa in the same sample to 1 row, adding all abundances
# Make sure your abundance/taxonomy table has this structure! (if you go for genus level comparison)
data_crop = data.groupby(["kingdom", "phylum", "class", "order", "family", "genus", "sample_id"]).abundance.sum().reset_index()

In [ ]:
data_crop

In [ ]:
print(data_crop.head())
data_crop.to_csv(f"{pro_ver}_data_crop.csv", sep=';', index=False)
print("Saved CSV of data crop.")

In [ ]:
data_crop

In [ ]:
import pandas as pd

#data_condition, only those conditions you want to compare. You dont always want to compare ALL samples you have
#                but instead only a few subgroups.
#data_crop, all taxonomy files and abundances in a 1D structure. Contains all samples, but unique ids were collapsed to
#           genus level and abundances were added.

# Initialisieren des neuen DataFrame data_crop2 mit denselben Spalten wie data_crop, aber leer
data_crop2 = pd.DataFrame(columns=data_crop.columns)

# Durchlaufen aller Einträge in der Spalte 'sample' von data_condition
for lookup_sample in data_condition['sample']:
    # Suchen der Einträge in data_crop, die dem Wert in lookup_sample entsprechen
    matches = data_crop[data_crop['sample_id'] == lookup_sample]
    
    # Hinzufügen der gefundenen Zeilen zum DataFrame data_crop2
    data_crop2 = pd.concat([data_crop2, matches])

    data_crop2.to_csv(f"{pro_ver}_data_crop2.csv", sep=';', index=False)
print("Saved CSV of data crop2.")
# Anzeigen des resultierenden DataFrame
data_crop2


In [ ]:
#3/13 - Build model folder #############################################################################################
#############################################################################################
## May take longer!
# Creates the sample.pickle for each sample in the folder defined above

from micom.workflows import build

manifest = build(data_crop2, out_folder=model_folder, model_db=test_db, cutoff=cutoff_value, threads=cpus_used, solver="cplex")
#manifest = build(data, out_folder=model_folder, model_db=test_db, cutoff=cutoff_value, threads=cpus_used)

print("manifest:", manifest)


In [ ]:
manifest

In [ ]:
#############################################################################################
#############################################################################################
#############################################################################################
from micom.workflows import tradeoff

tradeoff_rates = tradeoff(manifest, model_folder=model_folder, medium=medium, threads=cpus_used)
tradeoff_rates.head()



In [ ]:
# SAVING tradeoff data
import pickle

# CSV:
print(tradeoff_rates.head())
tradeoff_rates.to_csv(f"{pro_ver}_tradeoff.csv", sep=';', index=False)
print("Saved CSV of tradeoff data")

# PICKLE:
data_tuple = tuple(tradeoff_rates.itertuples(index=False, name=None))
with open(f'{pro_ver}_tradeoff.pickle', 'wb') as f:
    pickle.dump(data_tuple, f)
print("Saved pickle of tradeoff data")


In [ ]:
# QC, just to make sure the pickle file can be read again, in case you want to enter pipeline at this point

with open(f'{pro_ver}_tradeoff.pickle', 'rb') as f:
    loaded_tuple = pickle.load(f)

# Umwandlung des Tupels in einen DataFrame
# Usually stays like this. Reloading pickle of tradeoff manifest, defining the columns for the pickle
columns = ['abundance', 'growth_rate', 'reactions', 'metabolites', 'taxon', 'tradeoff', 'sample_id']
tradeoff_rates_df = pd.DataFrame(loaded_tuple, columns=columns)

# Anzeigen der ersten Zeilen des DataFrames
print(tradeoff_rates_df.head())


In [ ]:
#############################################################################################
tradeoff_rates.groupby("tradeoff").apply(
    lambda df: (df.growth_rate > 1e-6).sum()).reset_index()



In [ ]:
from micom.viz import plot_tradeoff

pl = plot_tradeoff(tradeoff_rates, filename=f"{pro_ver}_tradeoff.html")

print(f"Tradeoff of {pro_ver} visualized.")

In [ ]:
# Determine tradeoff. 

In [ ]:
tradeoff_value = float(1.0)
print("tradeoff_value: ", tradeoff_value)
print("---------------------------------------------------------")


In [ ]:
#4/13 - grow function #############################################################################################
#############################################################################################
## May take longer!

# Simulates the microbiome!

from micom.workflows import grow

res = grow(manifest, model_folder=model_folder, medium=medium, tradeoff=tradeoff_value, threads=cpus_used)


In [ ]:
### Log-Info########
import os
print(os.getcwd())
from datetime import datetime
# aktuelles Datum und Uhrzeit
now = datetime.now()
print("Aktuelles Datum und Uhrzeit:", now)
### Log-Info End########


import pandas as pd
import pickle

# Speichere das frische Tuple aus "res" in einer Pickle-Datei
with open(f"{pro_ver}_results.pickle", "wb") as f:
    pickle.dump(res, f)

# Lade das Tuple aus der Pickle-Datei
with open(f"{pro_ver}_results.pickle", "rb") as f:
    loaded_pickle_tuple = pickle.load(f)

# Überprüfe das geladene Tuple - es sollte genauso aussehen wie "res"
print(loaded_pickle_tuple)

#### ZUSÄTZLICH ALS REINE CSV-Information, um mal reinzuschauen
#### Kann NICHT wieder eingeladen werden
df = pd.DataFrame(res[0])
df.to_csv(pro_ver+'_res0.csv', index=True, header=True, sep=';')
df = pd.DataFrame(res[1])
df.to_csv(pro_ver+'_res1.csv', index=True, header=True, sep=';')
df = pd.DataFrame(res[2])
df.to_csv(pro_ver+'_res2.csv', index=True, header=True, sep=';')

### Log-Info########
import os
print(os.getcwd())
from datetime import datetime
# aktuelles Datum und Uhrzeit
now = datetime.now()
print("Aktuelles Datum und Uhrzeit:", now)
### Log-Info End########
    

In [ ]:
data_condition

In [ ]:
#10/13 - Load the two chosen conditions ########################################################
# First column contains the "sample id", read it into one column dataframe
data_condition_samp = data_condition[[data_condition.columns[0]]]
print(data_condition_samp)

# Second column contains the "condition", read it into one column dataframe
data_condition_col = data_condition[[data_condition.columns[1]]]
print(data_condition_col)

#A little hack - store it as csv file in the base jupyter folder
df = pd.DataFrame(data_condition_col)
df.to_csv(f'{pro_ver}_temp_condition_only.csv', index=False, header=True)

df = pd.DataFrame(data_condition_samp)
df.to_csv(f'{pro_ver}_temp_sample_only.csv', index=False, header=True)

# Read each CSV file as DataFrame
meta_sample_path = f'{pro_ver}_temp_sample_only.csv'
meta_sample = pd.read_csv(meta_sample_path, header=0)
# Manually convert DataFrame to Series if it contains only one column
if meta_sample.shape[1] == 1:
    meta_sample = meta_sample.iloc[:, 0]
print("meta_sample:", meta_sample)

meta_condition_path = f'{pro_ver}_temp_condition_only.csv'
meta_condition = pd.read_csv(meta_condition_path, header=0)
# Manually convert DataFrame to Series if it contains only one column
if meta_condition.shape[1] == 1:
    meta_condition = meta_condition.iloc[:, 0]
print("meta_condition:", meta_condition)

# Merge the two Series by setting the index of the 'meta_condition' Series to the values of the 'meta_sample' Series
meta_condition.index = meta_sample
print("meta_condition NEW:", meta_condition)


In [ ]:
from micom.measures import production_rates

#production_rate:
print("---> Performing production_rates() with:", pro_ver)
prod_rates = production_rates(res)
ind_prod_rates_path = f"{pro_ver}_prod_rates.csv"


df = pd.DataFrame(prod_rates)
df.to_csv(ind_prod_rates_path, index=True, header=True)
print("---- FINISHED production_rates()")


In [ ]:
# Modifying production_rate:
modified_prod_rates = prod_rates.copy()

# Erstellen einer neuen Spalte 'conditions' in modified_prod_rates
modified_prod_rates['conditions'] = ''

# Durchgehen jeder Zeile in modified_prod_rates
for index, row in modified_prod_rates.iterrows():
    # Finden der Übereinstimmung über die samples (TK_016)
    match = data_condition[data_condition['sample'] == row['sample_id']]

    # Wenn eine Übereinstimmung gefunden wurde, wird der Wert übertragen
    if not match.empty:
        modified_prod_rates.at[index, 'conditions'] = match['condition'].values[0]

ind_modprod_path = f"{pro_ver}_mod_prod_rate.csv"
# Speichern des modified_prod_rates DataFrame als CSV-Datei
modified_prod_rates.to_csv(ind_modprod_path, index=False, sep=";")
print(f'Created {ind_modprod_path}')

In [ ]:
modified_prod_rates

In [ ]:
from micom.stats import compare_groups
#compare_groups:

ind_comp_group_path = f"{pro_ver}_compare_groups.csv"
print("---> Performing compare_groups() with:", ind_comp_group_path)

tests = compare_groups(modified_prod_rates, "conditions", groups=["d13_control", "d13_CHX207"])  
# "group" ist der name der Spalte mit den Konditionen
# groups=["healthy", "cachexia"] 2
# d01_chx erster hit
# d13_chx zweiter hit
# groups=["d01_chx", "d13_chx"]
# log2_fold_change positiv --> gruppe ZWEI = d13_chx CHX erhöht.
# log2_fold_change negativ --> gruppe EINS = d01_chx gesund erhöht.

tests.sort_values(by="q")
tests.to_csv(ind_comp_group_path, index=False, sep=";")
print(f"---- FINISHED compare_groups() with {ind_comp_group_path}")


In [ ]:
import os
import pandas as pd
import glob

from bs4 import BeautifulSoup
from fnmatch import fnmatch

print(f'filepath_allmetab = {filepath_allmetab}')
      
metab_df = pd.read_csv(filepath_allmetab, sep=";")

# Erstelle ein Wörterbuch zur Abbildung der id_metab/id zu metabolite/name
replacement_dict = pd.Series(metab_df.name.values, index=metab_df.id).to_dict()

      



In [ ]:
replacement_dict

In [ ]:
change_id_filename = f"{pro_ver}_compare_groups.csv"
print("----> Screening CSV:", change_id_filename)
# Lese die Datei als DataFrame
df = pd.read_csv(os.path.join(working_dir, change_id_filename))
# Ersetze die alten Metaboliten-IDs durch die neuen
df.replace(replacement_dict, regex=True, inplace=True)

# ----> INSERT: Saving modified CSV in a desired folder:
df.to_csv(os.path.join(working_dir, 'mIDs_' + change_id_filename), index=False)
print("---- mIDs-modified CSV written.", 'mIDs_' + change_id_filename)

In [ ]:
change_id_filename = f"{pro_ver}_res0.csv"
print("----> Screening CSV:", change_id_filename)
# Lese die Datei als DataFrame
df = pd.read_csv(os.path.join(working_dir, change_id_filename))
# Ersetze die alten Metaboliten-IDs durch die neuen
df.replace(replacement_dict, regex=True, inplace=True)

# ----> INSERT: Saving modified CSV in a desired folder:
df.to_csv(os.path.join(working_dir, 'mIDs_' + change_id_filename), index=False)
print("---- mIDs-modified CSV written.", 'mIDs_' + change_id_filename)

In [ ]:
change_id_filename = f"{pro_ver}_res1.csv"
print("----> Screening CSV:", change_id_filename)
# Lese die Datei als DataFrame
df = pd.read_csv(os.path.join(working_dir, change_id_filename))
# Ersetze die alten Metaboliten-IDs durch die neuen
df.replace(replacement_dict, regex=True, inplace=True)

# ----> INSERT: Saving modified CSV in a desired folder:
df.to_csv(os.path.join(working_dir, 'mIDs_' + change_id_filename), index=False)
print("---- mIDs-modified CSV written.", 'mIDs_' + change_id_filename)

# Script Micom run2
Finished!